<a href="https://colab.research.google.com/github/Polticsmax/RAG-for-wikipedia/blob/main/RAG_for_wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I'll add a cell to install the `wikipedia` library.

In [1]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=1529ce35d76550d4a5c4ab22370c207ef490506d96cc3107ec68e69b6f0bc0ad
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


I'll add a cell to install the `faiss` library.

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.0 MB/s eta 0:00:00


In [3]:

import numpy as np

In [4]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from sentence_transformers import SentenceTransformer


In [10]:
import wikipedia

def get_wikipedia_content(topic):
    try:
        page = wikipedia.page(topic)
        return page.content
    except wikipedia.exceptions.PageError:
        return None
    except wikipedia.exceptions.DisambiguationError as e:
        # handle cases where the topic is ambiguous
        print(f"Ambiguous topic. Please be more specific. Options: {e.options}")
        return None

# user input
topic = input("Enter a topic to learn about: ")
document = get_wikipedia_content(topic)

if not document:
    print("Could not retrieve information.")
    exit()

Enter a topic to learn about: BJP


In [11]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")

def split_text(text, chunk_size=256, chunk_overlap=20):
    tokens = tokenizer.tokenize(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + chunk_size, len(tokens))
        chunks.append(tokenizer.convert_tokens_to_string(tokens[start:end]))
        if end == len(tokens):
            break
        start = end - chunk_overlap
    return chunks

chunks = split_text(document)
print(f"Number of chunks: {len(chunks)}")

Token indices sequence length is longer than the specified maximum sequence length for this model (9020 > 512). Running this sequence through the model will result in indexing errors


Number of chunks: 39


In [12]:
import faiss
embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
embeddings = embedding_model.encode(chunks)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

In [13]:
query = input("Ask a question about the topic: ")
query_embedding = embedding_model.encode([query])

k = 3
distances, indices = index.search(np.array(query_embedding), k)
retrieved_chunks = [chunks[i] for i in indices[0]]
print("Retrieved chunks:")
for chunk in retrieved_chunks:
    print("- " + chunk)

Ask a question about the topic: bjp 2024 election results
Retrieved chunks:
- won 282 seats, leading the nda to a tally of 336 seats in the 543 - seat lok sabha. narendra modi was sworn in as the 14th prime minister of india on 26 may 2014. the vote share of the bjp was 31 % of all votes cast, a low figure relative to the number of seats it won. this was the first instance since 1984 of a single party achieving an outright majority in the indian parliament and the first time that it achieved a majority in the lok sabha on its own strength. support was concentrated in the hindi - speaking belt in northcentral india. the magnitude of the victory was not predicted by most opinion and exit polls. political analysts have suggested several reasons for this victory, including the popularity of modi, and the loss of support for the congress due to the corruption scandals in its previous term. the bjp was also able to expand its traditionally upper - caste, upper - class support base and receiv

In [14]:
qa_model_name = "deepset/roberta-base-squad2"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)
qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer)

context = " ".join(retrieved_chunks)
answer = qa_pipeline(question=query, context=context)
print(f"Answer: {answer['answer']}")

Device set to use cpu


Answer: failure to reach out to rural indians
